In [1]:
from numpy.fft import fft, ifft, hfft
from scipy.signal import lfilter
from scipy.io import wavfile
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from IPython.display import Audio
from scipy import signal
import import_ipynb
import Helpers

figsize=(10,5)
linewidth=2.5
matplotlib.rcParams.update({'font.size': 16})

importing Jupyter notebook from Helpers.ipynb


In [ ]:
fs = 44100
time = np.linspace(0, 2, 2*fs)
samples = np.arange(2*fs)
fourier = np.linspace(0, .2*np.pi, 2*fs)

# Digital and additive oscillator

The first step of designing a synthesizer is to have oscillators. Here we write down 4 functions for generating each of the 4 basic waveforms. Each functions is given is 2 version, the digital version which simply generates the waveform using any mathematical function, and the additive version whoch uses a sum of sines to approximation the waveform.

## Sine wave

In [ ]:
def sine(A, w, phi, n):
    return A * np.sin(w*n - phi)

In [ ]:
plt.figure(figsize=figsize)
plt.plot(time, sine(1, 5, 0, time), linewidth=linewidth,color="slateblue",  label="Sine")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(loc='upper right')
plt.savefig("sine.pdf" ,bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(fourier[:1000], np.abs(fft(sine(1, 500, 0, time)))[:1000], linewidth=linewidth,color="slateblue",  label="Sine")
plt.xlabel("Angular frequency")
plt.ylabel("Log amplitude")
plt.yscale("log")
plt.legend(loc='upper right')
plt.savefig("sine_fft.pdf",bbox_inches='tight')
plt.show()

In [ ]:
sine1 = sine(1, 0.05, 0, samples)
wavfile.write('samples/sine.wav', fs, (0x7FFF * sine1).astype(np.int16))
Audio("samples/sine.wav", autoplay=False)

## Square wave

In [ ]:
def square(A, w, phi, n):
    return A*np.sign( np.sin( w*n - phi))

def square_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(1, order):
        out += np.sin((2*k-1)*w*n - phi)/(2*k-1)
        
    return 4*A/np.pi * out

In [ ]:
plt.figure(figsize=figsize)
plt.plot(time, square_sin(1, 5, 0, time, order=11), linewidth=linewidth, color="orange", label="Additive square")
plt.plot(time, square(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital square")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(loc='upper right')
plt.savefig("square.pdf",bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(fourier[:1000], np.abs(fft(square(1, 50, 0, time)))[:1000], linewidth=linewidth,color="slateblue",  label="Digital square")
plt.plot(fourier[:1000], np.abs(fft(square_sin(1, 50, 0, time, order=11)))[:1000], linewidth=linewidth, color="orange", label="Additive square")
plt.xlabel("Angular frequency")
plt.ylabel("Log amplitude")
plt.yscale("log")
plt.legend(loc='upper right')
plt.savefig("square_fft.pdf",bbox_inches='tight')
plt.show()

In [ ]:
square1 = square(1, 0.05, 0, samples)
wavfile.write('samples/square.wav', fs, (0x7FFF * square1).astype(np.int16))
Audio("samples/square.wav", autoplay=False)

## Triangle wave

In [ ]:
def triangle(A, w, phi, n):
    return 2*A/np.pi * np.arcsin( np.sin( w*n - phi))

def triangle_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(order):
        out += (-1)**k/(2*k+1)**2 * np.sin( (2*k+1)* w*n - phi )
        
    return 8*A/np.pi**2 * out

In [ ]:
plt.figure(figsize=figsize)
plt.plot(time, triangle_sin(1, 5, 0, time, order=2), linewidth=linewidth, color="orange", label="Additive triangle")
plt.plot(time, triangle(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital triangle")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(loc='upper right')
plt.savefig("triangle.pdf",bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(fourier[:1000], np.abs(fft(triangle(1, 50, 0, time)))[:1000], linewidth=linewidth, color="slateblue", label="Digital triangle")
plt.plot(fourier[:1000], np.abs(fft(triangle_sin(1, 50, 0, time, order=2)))[:1000], linewidth=linewidth, color="orange", label="Additive triangle")
plt.xlabel("Angular frequency")
plt.ylabel("Log amplitude")
plt.yscale("log")
plt.legend(loc='upper right')
plt.savefig("triangle_fft.pdf",bbox_inches='tight')
plt.show()

In [ ]:
tri1 = triangle(1, 0.05, 0, samples)
wavfile.write('samples/triangle.wav', fs, (0x7FFF * tri1).astype(np.int16))
Audio("samples/triangle.wav", autoplay=False)

## Sawtooth wave

In [ ]:
def saw(A, w, phi, n):
    return 2*A/np.pi * np.arctan( np.tan( w*n/2 - phi))

def saw_sin(A, w, phi, n, order=20):
    out = 0
    for k in range(1, order):
        out += (-1)**k/k * np.sin( k*w*n - phi )
        
    return - 2* A/np.pi * out

In [ ]:
plt.figure(figsize=figsize)
plt.plot(time, saw_sin(1, 5, 0, time, order=16), linewidth=linewidth, color="orange", label="Additive saw")
plt.plot(time, saw(1, 5, 0, time), linewidth=linewidth, color="slateblue", label="Digital saw")
plt.xlabel("Time")
plt.ylabel("Amplitude")
plt.legend(loc='upper right')
plt.savefig("saw.pdf",bbox_inches='tight')
plt.show()

In [ ]:
plt.figure(figsize=figsize)
plt.plot(fourier[:1000], np.abs(fft(saw(1, 50, 0, time)))[:1000], linewidth=linewidth, color="slateblue", label="Digital saw")
plt.plot(fourier[:1000], np.abs(fft(saw_sin(1, 50, 0, time, order=16)))[:1000], linewidth=linewidth, color="orange", label="Additive saw")
plt.xlabel("Angular frequency")
plt.ylabel("Log amplitude")
plt.yscale("log")
plt.legend(loc='upper right')
plt.savefig("saw_fft.pdf",bbox_inches='tight')
plt.show()

In [ ]:
saw1 = saw(1, 0.05, 0, samples)
wavfile.write('samples/saw.wav', fs, (0x7FFF * saw1).astype(np.int16))
Audio("samples/saw.wav", autoplay=False)

# FM synthesis

Alternatively to the subtractive and additive syntheses, one can use FM synthesis. Here, a modulator function takes as argument an arbitrary signal, as well as the parameters of a sine wave. The modulator modulates the sine wave with the signal. 

In [ ]:
def modulator(cw, cA, mod_signal, mod_index=1, oversampling_rate=8):
    """
    Block modulating a sine carrier with 
    - ang frequency cw, and 
    - amplitude cA, with the 
    - arbitrary signal mod_signal.
    Both must have the same length.
    """
    return np.array([cA * np.sin(cw*i + mod_index*2.*np.pi*mod_signal[i]) for i in range(mod_signal.size)])

In [ ]:
sine1 = sine(A=1, w=0.06, phi=-1, n=np.arange(2*fs, dtype=np.float32))
plt.figure(figsize=figsize)
plt.plot(sine1[:600])
wavfile.write('samples/sine1.wav', fs, (0x7FFF * sine1).astype(np.int16))
Audio("samples/sine1.wav", autoplay=False)

In [ ]:
sine2 = modulator(cw=0.0599, cA=1, mod_signal=sine1, mod_index=1.12)
plt.figure(figsize=figsize)
plt.plot(sine2[:600])
wavfile.write('samples/sine2.wav', fs, (0x7FFF * sine2).astype(np.int16))
Audio("samples/sine2.wav", autoplay=False)

Of course, a modulated signal can be used as the input of another modulator and so forth.

In [ ]:
sine3 = modulator(.000037, 1, sine2, mod_index=.3)
plt.figure(figsize=figsize)
plt.plot(sine3[:600])
wavfile.write('samples/sine3.wav', fs, (0x7FFF * sine3).astype(np.int16))
Audio("samples/sine3.wav", autoplay=False)

# Working with LFOs

As said in the class, an oscillator can be used to generate sound directly, as above, but also to tune any parameter of a synth. For instance, we can use an oscialltor as a LFO to tune another oscillator's amplitude over time, giving it a whobbly sound. Let's try!

In [ ]:
lfo1 = sine(1, 0.0005, 0, samples) # Low freqency oscillator
lfo_square1 = square(lfo1, 0.06, 0, samples) # LFO'ed sine wave

plt.figure(figsize=figsize)
plt.plot(lfo_square1[:5000])
wavfile.write('samples/lfo_square1.wav', fs, (0x7FFF * lfo_square1).astype(np.int16))
Audio("samples/lfo_square1.wav", autoplay=False)

One can also program the phase of an oscialltor with a LFO.

In [ ]:
lfo2 = sine(90, 0.0005, 0, samples) # Low freqency oscillator
lfo_sine2 = square(1, 0.06, lfo2, samples) # LFO'ed sine wave

plt.figure(figsize=figsize)
plt.plot(lfo_sine2[:6000])
wavfile.write('samples/lfo_sine2.wav', fs, (0x7FFF * lfo_sine2).astype(np.int16))
Audio("samples/lfo_sine2.wav", autoplay=False)